## Bring chemistry article data | citation data etc.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

import WOSutilities as wosutil

path2rawdata = '/mnt/disks/vault/wos2017-parsed/'

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
# we need to load articles and categories
# article_df = wosutil.load_wos_data(name = 'article', 
#                                    path2rawdata = path2rawdata,
#                                    year_list = None, 
#                                    columns = ['ArticleID', 'PubYear', 'Doctypes', 'Title','Journal', 'Journal Abbreviation ISO'], 
#                                    dropna = ['ArticleID', 'PubYear','Title'], 
#                                    duplicate_subset = ['ArticleID'],
#                                    isindict = {'Doctypes':np.sort(['Article','Letter','Review','Note'])}, 
#                                    verbose = 50)


article_df_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/articles.pql"
article_df = pd.read_pickle(article_df_path)

In [5]:
# article_df.to_pickle("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/articles.pql")

In [6]:
# select all categories for all the aricles published
# 

# category_list2 = np.array(['Physics, Multidisciplinary','Physics, Condensed Matter','Physics, Nuclear','Physics, Particles & Fields','Physics, Mathematical'])
# category_list3 = np.array(['Chemistry, Applied','Chemistry, Inorganic & Nuclear','Chemistry, Organic','Chemistry, Physical'])
# category_list4 = np.array(['Biophysics','Biochemistry & Molecular Biology',
#                            'Biochemical Research Methods'])

In [ ]:
category_list1 = np.array(['Physics, Applied'])
category_df1 = wosutil.load_wos_data(name = 'category', 
                                    path2rawdata = path2rawdata,
                                    year_list = None,
                                    columns = ['ArticleID', 'Type', 'Label'], 
                                    dropna = ['ArticleID'], 
                                    duplicate_subset = None,
                                    isindict = category_list1,
                                    verbose = 100)

100
200
300
400


In [ ]:
category_df1.to_pickle("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/phy1.pql")

In [ ]:
category_list1 = np.array(['Physics, Fluids & Plasmas'])
category_df2 = wosutil.load_wos_data(name = 'category', 
                                    path2rawdata = path2rawdata,
                                    year_list = None,
                                    columns = ['ArticleID', 'Type', 'Label'], 
                                    dropna = ['ArticleID'], 
                                    duplicate_subset = category_list1,
                                    isindict = category_list1,
                                    verbose = 100)
category_df2.to_pickle("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/phy2.pql")

In [ ]:
category_list1 = np.array(['Physics, Atomic, Molecular & Chemical'])
category_df3 = wosutil.load_wos_data(name = 'category', 
                                    path2rawdata = path2rawdata,
                                    year_list = None,
                                    columns = ['ArticleID', 'Type', 'Label'], 
                                    dropna = ['ArticleID'], 
                                    duplicate_subset = category_list1,
                                    isindict = category_list1,
                                    verbose = 100)
category_df3.to_pickle("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/phy3.pql")

In [ ]:
category_list1 = np.array(['Physics, Multidisciplinary'])
category_df4 = wosutil.load_wos_data(name = 'category', 
                                    path2rawdata = path2rawdata,
                                    year_list = None,
                                    columns = ['ArticleID', 'Type', 'Label'], 
                                    dropna = ['ArticleID'], 
                                    duplicate_subset = category_list1,
                                    isindict = category_list1,
                                    verbose = 100)

In [ ]:
category_list1 = np.array(['Physics, Condensed Matter'])
category_df5 = wosutil.load_wos_data(name = 'category', 
                                    path2rawdata = path2rawdata,
                                    year_list = None,
                                    columns = ['ArticleID', 'Type', 'Label'], 
                                    dropna = ['ArticleID'], 
                                    duplicate_subset = category_list1,
                                    isindict = category_list1,
                                    verbose = 100)

In [ ]:
category_list1 = np.array(['Physics, Physics, Nuclear'])
category_df6 = wosutil.load_wos_data(name = 'category', 
                                    path2rawdata = path2rawdata,
                                    year_list = None,
                                    columns = ['ArticleID', 'Type', 'Label'], 
                                    dropna = ['ArticleID'], 
                                    duplicate_subset = category_list1,
                                    isindict = category_list1,
                                    verbose = 100)

In [ ]:
physics_cat_df = pd.concat([category_df1, 
                            category_df2, 
                            category_df3, 
                            category_df4,
                            category_df5,
                            category_df6], ignore_index=True)
physics_cat_df.to_pickle("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/physics_categories.pql")

In [ ]:
# How many article ids are present in category_df
print(len(category_df['ArticleID'].unique()))

# Are there NaN values in article id
print(category_df.isnull().values.any())

# Are there NaN values in article id
print(set(category_df['ArticleID'].isna()))

# Are there NaN values in Label
print(set(category_df['Label'].isna()))

53671796
False
{False}


In [ ]:
# Checking articles dataframe
print(len(article_df['ArticleID'].unique()))

# Are there NaN values in article id
print(article_df.isnull().values.any())

39171585
False


In [ ]:
# joining citations with articles and categories

# joining the dataframes
data = None
data = category_df.merge(article_df, 
                         how = 'inner',
                         on = 'ArticleID')

In [ ]:
data.count()

ArticleID                   406341980
Type                        406341980
Label                       406341980
PubYear                     406341980
Doctypes                    406341980
Title                       406341980
Journal                     406341980
Journal Abbreviation ISO    406341980
dtype: int64

In [ ]:
data = data.drop_duplicates()
data.to_pickle("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/articles_and_categories_phy_chem_bio.pql")